In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, display_markdown
import pandas as pd

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [2]:
rerun = False

In [3]:
### Tools' setting ###
# PATHS
owl_version = '1.1.0'
owl_bin     = 'owl-'+owl_version+'/bin/'
make_tgba   = 'ltl2tgba --deterministic -f %f' 
# Tools
owl         = owl_bin+'ltl2ldba %[MWi]f'
nba2ldba    = make_tgba + ' --ba | '+owl_bin+'nba2ldba'
seminator   = make_tgba + ' | ./seminator'
cy          = make_tgba + ' | ./seminator --via-tba'
# Options
cut         = ' --cd'
simp_sd     = ' | autfilt --small --tgba'
simp_cd     = ' | autfilt -x simul=1,ba-simul=1 --small --tgba'
nos         = ' -s0' # disables Spot's simplifications used in Seminator
end         = ' > %O' # saves result to file

### Ltlcross runner configuration ###
tools = {## Cut-deterministic
         'cd.ltl2ldba.no'     : owl + end,
         'cd.ltl2ldba.yes'    : owl + simp_cd + end,
         'cd.seminator.no'    : seminator + nos + cut + end,
         'cd.seminator.yes'   : seminator + cut + end,
         'cd.2-step.no'       : cy + nos + cut + end,     
         'cd.2-step.yes'      : cy + cut + end,
         ### Semi-deterministic
         'sd.ltl2ldba.no'     : owl + ' -n' + end,
         'sd.ltl2ldba.yes'    : owl + ' -n' + simp_sd + end,
         'sd.nba2ldba.no'     : nba2ldba + end,
         'sd.nba2ldba.yes'    : nba2ldba + simp_sd + end,
         'sd.seminator.no'    : seminator + nos + end,
         'sd.seminator.yes'   : seminator + end,
         'sd.2-step.no'       : cy + nos + end,  
         'sd.2-step.yes'      : cy + end,
        }
sd_tools = ['sd.seminator.no','sd.seminator.yes',
            'sd.2-step.no','sd.2-step.yes',
            'sd.ltl2ldba.no','sd.ltl2ldba.yes',
            'sd.nba2ldba.no','sd.nba2ldba.yes']
cd_tools = ['cd.seminator.no','cd.seminator.yes',
            'cd.2-step.no','cd.2-step.yes',
            'cd.ltl2ldba.no','cd.ltl2ldba.yes']
### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']
tool_order = ['seminator','2-step','ltl2ldba','nba2ldba']

In [4]:
runners = {}
for source in ('literature','random'):
    for t in ('det','cd','sd','nd'):
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)
        if rerun:
            runners[name].run_ltlcross(timeout='120',check=False)
        runners[name].parse_results()

In [5]:
def get_counts(runner):
    v = r.values.states.copy()
    v.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in v.columns])
    return v.cd.dropna().count()[0], v.sd.dropna().count()[0]

## Cummulative results

Gather the data into one big DataFrame

In [6]:
data = pd.DataFrame()
for (name,r) in runners.items():
    # Divide into sd_tools & cd_tools to treat timeouts separately.
    tmp_cd = pd.DataFrame(r.cummulative(tools=cd_tools),columns=[name])
    tmp_sd = pd.DataFrame(r.cummulative(tools=sd_tools),columns=[name])
    data = data.append(tmp_cd.append(tmp_sd).transpose())
data

tool,cd.seminator.no,cd.seminator.yes,cd.2-step.no,cd.2-step.yes,cd.ltl2ldba.no,cd.ltl2ldba.yes,sd.seminator.no,sd.seminator.yes,sd.2-step.no,sd.2-step.yes,sd.ltl2ldba.no,sd.ltl2ldba.yes,sd.nba2ldba.no,sd.nba2ldba.yes
literature_nd,656,414,698,450,470,410,472,369,514,404,666,376,869,573
literature_sd,13,13,13,13,53,40,13,13,13,13,41,17,17,13
literature_det,556,556,585,585,1039,809,556,556,585,585,1277,855,600,600
random_sd,734,712,735,713,1495,1275,704,704,705,705,1228,784,850,774
random_det,413,413,414,414,570,497,413,413,414,414,638,446,426,426
random_cd,463,463,463,463,732,649,463,463,463,463,733,539,863,634
random_nd,2025,1140,2103,1183,1387,1038,1231,937,1274,987,1314,804,3657,1875
literature_cd,194,194,198,198,612,488,194,194,198,198,838,341,377,240


Get the number of formulas such that all tools finished within timeout for each category.

In [7]:
counts = pd.DataFrame(index=pd.Index([],name='source'),columns=['cd..n','sd..n'])
for (name,r) in runners.items():
    cd_c, sd_c = get_counts(r)
    counts = counts.append(pd.DataFrame({'cd..n':cd_c,'sd..n':sd_c},index=[name]))
counts

,cd..n,sd..n
literature_nd,23,23
literature_sd,3,3
literature_det,149,149
random_sd,100,100
random_det,100,100
random_cd,100,100
random_nd,100,100
literature_cd,46,46


Converts the DataFrame to use MultiIndices for rows and columns

In [8]:
from pandas.api.types import CategoricalDtype
automata_type = CategoricalDtype(categories=['det','cd','sd','nd'], ordered=True)
source_type = CategoricalDtype(categories=['random', 'literature', 'rand. (T/O)', 'lit. (T/O)'], ordered=True)

In [9]:
def make_hierarchical(data):
    # Split index into origin & ltl2tgba's output type
    df = data.copy()
    df.index.name = 'origin'
    df.reset_index(inplace=True)
    df['type'] = df.origin.apply(lambda x: x.split('_')[1]).astype(automata_type)
    df['origin'] = df.origin.apply(lambda x: x.split('_')[0]).astype(source_type)
    df = df.set_index(['origin','type']).sort_index()
    # Split columns in tool & reductions
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in df.columns])
    return df

In [10]:
tmp = make_hierarchical(data.join(counts)).sort_index(axis=1)
tmp

cd                                               sd         \
                     2-step       ltl2ldba       seminator            2-step   
                   n     no   yes       no   yes        no   yes    n     no   
origin     type                                                                
random     det   100    414   414      570   497       413   413  100    414   
           cd    100    463   463      732   649       463   463  100    463   
           sd    100    735   713     1495  1275       734   712  100    705   
           nd    100   2103  1183     1387  1038      2025  1140  100   1274   
literature det   149    585   585     1039   809       556   556  149    585   
           cd     46    198   198      612   488       194   194   46    198   
           sd      3     13    13       53    40        13    13    3     13   
           nd     23    698   450      470   410       656   414   23    514   

                                                                  
                     ltl2ldba      nba2ldba       seminator       
                 yes       no  yes       no   yes        no  yes  
origin     type                                                   
random     det   414      638  446      426   426       413  413  
           cd    463      733  539      863   634       463  463  
           sd    705     1228  784      850   774       704  704  
           nd    987     1314  804     3657  1875      1231  937  
literature det   585     1277  855      600   600       556  556  
           cd    198      838  341      377   240       194  194  
           sd     13       41   17       17    13        13   13  
           nd    404      666  376      869   573       472  369

In [11]:
cd = tmp.cd[['']+tool_order]
sd = tmp.sd[['']+tool_order]

In [12]:
cd

seminator       2-step       ltl2ldba      
                   n        no   yes     no   yes       no   yes
origin     type                                                 
random     det   100       413   413    414   414      570   497
           cd    100       463   463    463   463      732   649
           sd    100       734   712    735   713     1495  1275
           nd    100      2025  1140   2103  1183     1387  1038
literature det   149       556   556    585   585     1039   809
           cd     46       194   194    198   198      612   488
           sd      3        13    13     13    13       53    40
           nd     23       656   414    698   450      470   410

In [13]:
sd

seminator      2-step      ltl2ldba      nba2ldba      
                   n        no  yes     no  yes       no  yes       no   yes
origin     type                                                             
random     det   100       413  413    414  414      638  446      426   426
           cd    100       463  463    463  463      733  539      863   634
           sd    100       704  704    705  705     1228  784      850   774
           nd    100      1231  937   1274  987     1314  804     3657  1875
literature det   149       556  556    585  585     1277  855      600   600
           cd     46       194  194    198  198      838  341      377   240
           sd      3        13   13     13   13       41   17       17    13
           nd     23       472  369    514  404      666  376      869   573

## Timeouts

In [14]:
TO_data = pd.DataFrame()
for (name,r) in runners.items():
    TO_data = TO_data.append(pd.DataFrame(r.compute_timeouts(),columns=[name]).transpose())
TO = make_hierarchical(TO_data).replace(0,pd.NaT).dropna(how='all').fillna(0)
TO

Empty DataFrame
Columns: [(cd, 2-step, no), (cd, 2-step, yes), (cd, ltl2ldba, no), (cd, ltl2ldba, yes), (cd, seminator, no), (cd, seminator, yes), (sd, 2-step, no), (sd, 2-step, yes), (sd, ltl2ldba, no), (sd, ltl2ldba, yes), (sd, nba2ldba, no), (sd, nba2ldba, yes), (sd, seminator, no), (sd, seminator, yes)]
Index: []

There is no timeout.

# Final results
We use the `display` function to propagate the results to notebooks that just run this one.

In [15]:
display_markdown('''### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.
''',raw=True)

### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.


In [16]:
display(cd)

seminator       2-step       ltl2ldba      
                   n        no   yes     no   yes       no   yes
origin     type                                                 
random     det   100       413   413    414   414      570   497
           cd    100       463   463    463   463      732   649
           sd    100       734   712    735   713     1495  1275
           nd    100      2025  1140   2103  1183     1387  1038
literature det   149       556   556    585   585     1039   809
           cd     46       194   194    198   198      612   488
           sd      3        13    13     13    13       53    40
           nd     23       656   414    698   450      470   410

In [17]:
display_markdown('''### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.
''',raw=True)

### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.


In [18]:
display(sd)

seminator      2-step      ltl2ldba      nba2ldba      
                   n        no  yes     no  yes       no  yes       no   yes
origin     type                                                             
random     det   100       413  413    414  414      638  446      426   426
           cd    100       463  463    463  463      733  539      863   634
           sd    100       704  704    705  705     1228  784      850   774
           nd    100      1231  937   1274  987     1314  804     3657  1875
literature det   149       556  556    585  585     1277  855      600   600
           cd     46       194  194    198  198      838  341      377   240
           sd      3        13   13     13   13       41   17       17    13
           nd     23       472  369    514  404      666  376      869   573